## 使用重复元素的网络(VGG)
$AlexNet$在$LeNet$的基础上增加了$3$个卷积层。但$AlexNet$作者对它们的卷积窗口、输出通道数和构造顺序均做了大量的调整。虽然$AlexNet$指明了深度卷积神经网络可以取得出色的结果，但并没有提供简单的规则以指导后来的研究者如何设计新的网络。我们将在本章的后续几节里介绍几种不同的深度网络设计思路。

本节介绍$VGG$，$VGG$提出了可以通过**重复使用简单的基础块**来构建深度模型的思路。

### VGG块
$VGG$块的组成规律是：连续使用数个相同的填充为$1$、窗口形状为$3\times 3$的卷积层后接上一个步幅为$2$、窗口形状为$2\times 2$的最大池化层。**卷积层保持输入的高和宽不变，而池化层则对其减半**。

对于给定的感受野（与输出有关的输入图片的局部大小），采用**堆积的小卷积核优于采用大的卷积核**，因为可以增加网络深度来保证学习更复杂的模式，而且代价还比较小（参数更少）。例如，在$VGG$中，使用了$3$个$3\times 3$卷积核来代替$7\times 7$卷积核，使用了$2$个$3\times 3$卷积核来代替$5\times 5$卷积核，这样做的主要目的是在保证具有相同感知野的条件下，提升了网络的深度，在一定程度上提升了神经网络的效果。

In [1]:
import time
import torch
from torch import nn, optim
import sys
sys.path.append("..")
import d2lzh_pytorch as d2l
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def vgg_block(num_convs, in_channels, out_channels):
    blk = []
    for i in range(num_convs):
        if i == 0:
            blk.append(nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1))
        else:
            blk.append(nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1))
    blk.append(nn.MaxPool2d(2))
    return nn.Sequential(*blk)

### VGG网络
与$AlexNet$和$LeNet$一样，$VGG$网络由卷积层模块后接全连接层模块构成。卷积层模块串联数个$vgg\_block$，其超参数由变量$conv\_arch$定义。该变量指定了每个$VGG$块里卷积层个数和输入输出通道数。全连接模块则跟$AlexNet$中的一样。

现在我们构造一个$VGG$网络。它有$5$个卷积块，前$2$块使用单卷积层，而后$3$块使用双卷积层。第一块的输入输出通道分别是$1$（因为下面要使用的$Fashion-MNIST$数据的通道数为$1$）和$64$，之后每次对输出通道数翻倍，直到变为$512$。因为这个网络使用了$8$个卷积层和$3$个全连接层，所以经常被称为$VGG-11$。

In [2]:
conv_arch = ((1, 1, 64), (1, 64, 128), (2, 128, 256), (2, 256, 512), (2, 512, 512))
# 经过5个vgg_block，宽高会减半五次，变成224/32=7
fc_features = 512*7*7
fc_hidden_units = 4096

In [3]:
def vgg(conv_arch, fc_features, fc_hidden_units=4096):
    net = nn.Sequential()
    for i, (num_convs, in_channels, out_channels) in enumerate(conv_arch):
        net.add_module("vgg_block_"+str(i+1), vgg_block(num_convs, in_channels,out_channels))
    # 全连接层
    net.add_module("fc", nn.Sequential(
        d2l.FlattenLayer(),
        nn.Linear(fc_features, fc_hidden_units),
        nn.ReLU(),
        nn.Dropout(0.5),
        nn.Linear(fc_hidden_units, fc_hidden_units),
        nn.ReLU(),
        nn.Dropout(0.5),
        nn.Linear(fc_hidden_units, 10)
    ))
    return net

下面构造一个高和宽均为$224$的单通道数据样本来观察每一层的输出形状。

In [4]:
net = vgg(conv_arch, fc_features, fc_hidden_units)
X = torch.rand(1, 1, 224, 224)
# named_children获取一级子模块及其名字(named_modules会返回所有子模块,包括子模块的子模块)
for name, blk in net.named_children():
    X = blk(X)
    print(name, 'output shape', X.shape)

vgg_block_1 output shape torch.Size([1, 64, 112, 112])
vgg_block_2 output shape torch.Size([1, 128, 56, 56])
vgg_block_3 output shape torch.Size([1, 256, 28, 28])
vgg_block_4 output shape torch.Size([1, 512, 14, 14])
vgg_block_5 output shape torch.Size([1, 512, 7, 7])
fc output shape torch.Size([1, 10])


可以看到，每次我们将输入的高和宽减半，直到最终高和宽变成$7$后传入全连接层。与此同时，输出通道数每次翻倍，直到变成$512$。因为每个卷积层的窗口大小一样，所以每层的模型参数尺寸和计算复杂度与输入高、输入宽、输入通道数和输出通道数的乘积成正比。$VGG$这种高和宽减半以及通道翻倍的设计使得多数卷积层都有相同的模型参数尺寸和计算复杂度。

### 获取数据和训练模型
因为$VGG-11$计算上比$AlexNet$更加复杂，出于测试的目的我们构造一个通道数更小，或者说更窄的网络在$Fashion-MNIST$数据集上进行训练。

In [5]:
ratio = 8
small_conv_arch = [
    (1, 1, 64//ratio),
    (1, 64//ratio, 128//ratio),
    (2, 128//ratio, 256//ratio),
    (2, 256//ratio, 512//ratio),
    (2, 512//ratio, 512//ratio)
]
net = vgg(small_conv_arch, fc_features//ratio, fc_hidden_units//ratio)
print(net)

Sequential(
  (vgg_block_1): Sequential(
    (0): Conv2d(1, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (vgg_block_2): Sequential(
    (0): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (vgg_block_3): Sequential(
    (0): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (vgg_block_4): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (vgg_block_5): Sequential(
    (0): Conv2d(64, 64, kernel_size=(3, 3), 

In [6]:
batch_size = 64
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size, resize=224)

lr, epochs = 0.001, 5
optimizer = torch.optim.Adam(net.parameters(), lr=lr)
d2l.train_ch5(net, train_iter, test_iter, batch_size, optimizer, device, epochs)

training on: cpu
step 1, train_acc: 0.1250
step 2, train_acc: 0.1641
step 3, train_acc: 0.1354
step 4, train_acc: 0.1250
step 5, train_acc: 0.1187
step 6, train_acc: 0.1224
step 7, train_acc: 0.1406
step 8, train_acc: 0.1719
step 9, train_acc: 0.1858
step 10, train_acc: 0.2125
step 11, train_acc: 0.2401
step 12, train_acc: 0.2539
step 13, train_acc: 0.2632
step 14, train_acc: 0.2679
step 15, train_acc: 0.2844
step 16, train_acc: 0.2959
step 17, train_acc: 0.3042
step 18, train_acc: 0.3177
step 19, train_acc: 0.3273
step 20, train_acc: 0.3367
step 21, train_acc: 0.3415
step 22, train_acc: 0.3480
step 23, train_acc: 0.3533
step 24, train_acc: 0.3607
step 25, train_acc: 0.3681
step 26, train_acc: 0.3738
step 27, train_acc: 0.3767
step 28, train_acc: 0.3783
step 29, train_acc: 0.3852
step 30, train_acc: 0.3875
step 31, train_acc: 0.3926
step 32, train_acc: 0.3984
step 33, train_acc: 0.4039
step 34, train_acc: 0.4090
step 35, train_acc: 0.4107
step 36, train_acc: 0.4145
step 37, train_acc: 

step 297, train_acc: 0.7310
step 298, train_acc: 0.7311
step 299, train_acc: 0.7314
step 300, train_acc: 0.7320
step 301, train_acc: 0.7322
step 302, train_acc: 0.7327
step 303, train_acc: 0.7329
step 304, train_acc: 0.7333
step 305, train_acc: 0.7338
step 306, train_acc: 0.7340
step 307, train_acc: 0.7344
step 308, train_acc: 0.7345
step 309, train_acc: 0.7348
step 310, train_acc: 0.7352
step 311, train_acc: 0.7353
step 312, train_acc: 0.7356
step 313, train_acc: 0.7358
step 314, train_acc: 0.7363
step 315, train_acc: 0.7366
step 316, train_acc: 0.7368
step 317, train_acc: 0.7367
step 318, train_acc: 0.7369
step 319, train_acc: 0.7373
step 320, train_acc: 0.7375
step 321, train_acc: 0.7378
step 322, train_acc: 0.7381
step 323, train_acc: 0.7385
step 324, train_acc: 0.7388
step 325, train_acc: 0.7388
step 326, train_acc: 0.7390
step 327, train_acc: 0.7393
step 328, train_acc: 0.7394
step 329, train_acc: 0.7397
step 330, train_acc: 0.7401
step 331, train_acc: 0.7403
step 332, train_acc:

step 590, train_acc: 0.7873
step 591, train_acc: 0.7874
step 592, train_acc: 0.7876
step 593, train_acc: 0.7877
step 594, train_acc: 0.7878
step 595, train_acc: 0.7878
step 596, train_acc: 0.7880
step 597, train_acc: 0.7881
step 598, train_acc: 0.7883
step 599, train_acc: 0.7884
step 600, train_acc: 0.7884
step 601, train_acc: 0.7887
step 602, train_acc: 0.7887
step 603, train_acc: 0.7888
step 604, train_acc: 0.7890
step 605, train_acc: 0.7892
step 606, train_acc: 0.7893
step 607, train_acc: 0.7895
step 608, train_acc: 0.7896
step 609, train_acc: 0.7897
step 610, train_acc: 0.7898
step 611, train_acc: 0.7897
step 612, train_acc: 0.7899
step 613, train_acc: 0.7900
step 614, train_acc: 0.7902
step 615, train_acc: 0.7903
step 616, train_acc: 0.7904
step 617, train_acc: 0.7905
step 618, train_acc: 0.7905
step 619, train_acc: 0.7907
step 620, train_acc: 0.7907
step 621, train_acc: 0.7908
step 622, train_acc: 0.7908
step 623, train_acc: 0.7910
step 624, train_acc: 0.7912
step 625, train_acc:

step 883, train_acc: 0.8109
step 884, train_acc: 0.8110
step 885, train_acc: 0.8111
step 886, train_acc: 0.8111
step 887, train_acc: 0.8112
step 888, train_acc: 0.8113
step 889, train_acc: 0.8113
step 890, train_acc: 0.8114
step 891, train_acc: 0.8114
step 892, train_acc: 0.8116
step 893, train_acc: 0.8117
step 894, train_acc: 0.8118
step 895, train_acc: 0.8119
step 896, train_acc: 0.8120
step 897, train_acc: 0.8120
step 898, train_acc: 0.8121
step 899, train_acc: 0.8121
step 900, train_acc: 0.8121
step 901, train_acc: 0.8123
step 902, train_acc: 0.8124
step 903, train_acc: 0.8125
step 904, train_acc: 0.8125
step 905, train_acc: 0.8125
step 906, train_acc: 0.8126
step 907, train_acc: 0.8127
step 908, train_acc: 0.8128
step 909, train_acc: 0.8128
step 910, train_acc: 0.8128
step 911, train_acc: 0.8129
step 912, train_acc: 0.8130
step 913, train_acc: 0.8130
step 914, train_acc: 0.8130
step 915, train_acc: 0.8130
step 916, train_acc: 0.8131
step 917, train_acc: 0.8131
step 918, train_acc:

step 239, train_acc: 0.8709
step 240, train_acc: 0.8710
step 241, train_acc: 0.8712
step 242, train_acc: 0.8712
step 243, train_acc: 0.8709
step 244, train_acc: 0.8705
step 245, train_acc: 0.8703
step 246, train_acc: 0.8707
step 247, train_acc: 0.8707
step 248, train_acc: 0.8707
step 249, train_acc: 0.8709
step 250, train_acc: 0.8709
step 251, train_acc: 0.8709
step 252, train_acc: 0.8707
step 253, train_acc: 0.8707
step 254, train_acc: 0.8708
step 255, train_acc: 0.8708
step 256, train_acc: 0.8706
step 257, train_acc: 0.8709
step 258, train_acc: 0.8709
step 259, train_acc: 0.8711
step 260, train_acc: 0.8712
step 261, train_acc: 0.8712
step 262, train_acc: 0.8714
step 263, train_acc: 0.8714
step 264, train_acc: 0.8717
step 265, train_acc: 0.8716
step 266, train_acc: 0.8719
step 267, train_acc: 0.8720
step 268, train_acc: 0.8722
step 269, train_acc: 0.8720
step 270, train_acc: 0.8720
step 271, train_acc: 0.8718
step 272, train_acc: 0.8718
step 273, train_acc: 0.8717
step 274, train_acc:

...

step 416, train_acc: 0.9069
step 417, train_acc: 0.9070
step 418, train_acc: 0.9070
step 419, train_acc: 0.9070
step 420, train_acc: 0.9071
step 421, train_acc: 0.9071
step 422, train_acc: 0.9070
step 423, train_acc: 0.9071
step 424, train_acc: 0.9071
step 425, train_acc: 0.9070
step 426, train_acc: 0.9071
step 427, train_acc: 0.9070
step 428, train_acc: 0.9069
step 429, train_acc: 0.9070
step 430, train_acc: 0.9069
step 431, train_acc: 0.9070
step 432, train_acc: 0.9071
step 433, train_acc: 0.9070
step 434, train_acc: 0.9069
step 435, train_acc: 0.9068
step 436, train_acc: 0.9069
step 437, train_acc: 0.9069
step 438, train_acc: 0.9068
step 439, train_acc: 0.9069
step 440, train_acc: 0.9069
step 441, train_acc: 0.9067
step 442, train_acc: 0.9066
step 443, train_acc: 0.9065
step 444, train_acc: 0.9065
step 445, train_acc: 0.9064
step 446, train_acc: 0.9063
step 447, train_acc: 0.9063
step 448, train_acc: 0.9061
step 449, train_acc: 0.9060
step 450, train_acc: 0.9060
step 451, train_acc:

step 709, train_acc: 0.9054
step 710, train_acc: 0.9054
step 711, train_acc: 0.9055
step 712, train_acc: 0.9055
step 713, train_acc: 0.9055
step 714, train_acc: 0.9055
step 715, train_acc: 0.9054
step 716, train_acc: 0.9054
step 717, train_acc: 0.9054
step 718, train_acc: 0.9053
step 719, train_acc: 0.9052
step 720, train_acc: 0.9052
step 721, train_acc: 0.9052
step 722, train_acc: 0.9053
step 723, train_acc: 0.9053
step 724, train_acc: 0.9052
step 725, train_acc: 0.9052
step 726, train_acc: 0.9051
step 727, train_acc: 0.9052
step 728, train_acc: 0.9052
step 729, train_acc: 0.9051
step 730, train_acc: 0.9053
step 731, train_acc: 0.9053
step 732, train_acc: 0.9053
step 733, train_acc: 0.9053
step 734, train_acc: 0.9053
step 735, train_acc: 0.9053
step 736, train_acc: 0.9053
step 737, train_acc: 0.9053
step 738, train_acc: 0.9054
step 739, train_acc: 0.9054
step 740, train_acc: 0.9053
step 741, train_acc: 0.9053
step 742, train_acc: 0.9053
step 743, train_acc: 0.9053
step 744, train_acc:

step 64, train_acc: 0.9070
step 65, train_acc: 0.9075
step 66, train_acc: 0.9065
step 67, train_acc: 0.9069
step 68, train_acc: 0.9069
step 69, train_acc: 0.9081
step 70, train_acc: 0.9083
step 71, train_acc: 0.9089
step 72, train_acc: 0.9080
step 73, train_acc: 0.9073
step 74, train_acc: 0.9073
step 75, train_acc: 0.9067
step 76, train_acc: 0.9069
step 77, train_acc: 0.9073
step 78, train_acc: 0.9073
step 79, train_acc: 0.9078
step 80, train_acc: 0.9074
step 81, train_acc: 0.9076
step 82, train_acc: 0.9082
step 83, train_acc: 0.9079
step 84, train_acc: 0.9079
step 85, train_acc: 0.9077
step 86, train_acc: 0.9081
step 87, train_acc: 0.9080
step 88, train_acc: 0.9084
step 89, train_acc: 0.9077
step 90, train_acc: 0.9073
step 91, train_acc: 0.9076
step 92, train_acc: 0.9081
step 93, train_acc: 0.9084
step 94, train_acc: 0.9086
step 95, train_acc: 0.9090
step 96, train_acc: 0.9089
step 97, train_acc: 0.9091
step 98, train_acc: 0.9096
step 99, train_acc: 0.9096
step 100, train_acc: 0.9097


step 358, train_acc: 0.9106
step 359, train_acc: 0.9106
step 360, train_acc: 0.9104
step 361, train_acc: 0.9104
step 362, train_acc: 0.9105
step 363, train_acc: 0.9106
step 364, train_acc: 0.9106
step 365, train_acc: 0.9107
step 366, train_acc: 0.9107
step 367, train_acc: 0.9106
step 368, train_acc: 0.9107
step 369, train_acc: 0.9106
step 370, train_acc: 0.9105
step 371, train_acc: 0.9106
step 372, train_acc: 0.9106
step 373, train_acc: 0.9104
step 374, train_acc: 0.9105
step 375, train_acc: 0.9106
step 376, train_acc: 0.9106
step 377, train_acc: 0.9105
step 378, train_acc: 0.9105
step 379, train_acc: 0.9106
step 380, train_acc: 0.9108
step 381, train_acc: 0.9108
step 382, train_acc: 0.9108
step 383, train_acc: 0.9109
step 384, train_acc: 0.9109
step 385, train_acc: 0.9109
step 386, train_acc: 0.9108
step 387, train_acc: 0.9110
step 388, train_acc: 0.9110
step 389, train_acc: 0.9109
step 390, train_acc: 0.9110
step 391, train_acc: 0.9109
step 392, train_acc: 0.9110
step 393, train_acc:

step 651, train_acc: 0.9119
step 652, train_acc: 0.9118
step 653, train_acc: 0.9118
step 654, train_acc: 0.9118
step 655, train_acc: 0.9117
step 656, train_acc: 0.9118
step 657, train_acc: 0.9118
step 658, train_acc: 0.9116
step 659, train_acc: 0.9118
step 660, train_acc: 0.9116
step 661, train_acc: 0.9116
step 662, train_acc: 0.9115
step 663, train_acc: 0.9115
step 664, train_acc: 0.9115
step 665, train_acc: 0.9115
step 666, train_acc: 0.9115
step 667, train_acc: 0.9115
step 668, train_acc: 0.9115
step 669, train_acc: 0.9115
step 670, train_acc: 0.9114
step 671, train_acc: 0.9114
step 672, train_acc: 0.9114
step 673, train_acc: 0.9114
step 674, train_acc: 0.9115
step 675, train_acc: 0.9115
step 676, train_acc: 0.9114
step 677, train_acc: 0.9114
step 678, train_acc: 0.9115
step 679, train_acc: 0.9115
step 680, train_acc: 0.9116
step 681, train_acc: 0.9116
step 682, train_acc: 0.9116
step 683, train_acc: 0.9116
step 684, train_acc: 0.9116
step 685, train_acc: 0.9116
step 686, train_acc:

+ VGG-11通过5个可以重复使用的卷积块来构造网络。根据每块里卷积层个数和输出通道数的不同可以定义出不同的VGG模型。